In [1]:
import pandas as pd
import geopandas as gpd
import sys
sys.path.append(r'C:\Users\mfpen\Documents\Repositorios\geoloc2\Detecciondeterrenos')
from codigos import obtener_curt

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\mfpen\AppData\Local\Temp\ipykernel_10252\2563911923.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthed

In [2]:
CURT=gpd.read_file(r"D:\Secretaria\Bases_shp_INEGI\CURT\098 Naucalpan\NAUCALPAN_PREDIOS.shp")
CURT

,cve_ent,cve_mun,cve_loc,id_cat,curt,notas,geometry
0,15,098,0000,0980845763000000,192937370009913582692,NaN,"POLYGON Z ((-99.23275 19.49368 0.00000, -99.23..."
1,15,098,0000,0980946128000000,192559270609916021438,NaN,"POLYGON Z ((-99.26718 19.43312 0.00000, -99.26..."
2,15,098,0000,0980332635000000,192612034409915107715,NaN,"POLYGON Z ((-99.25296 19.43671 0.00000, -99.25..."
3,15,098,0000,0980902232000000,192635463709916472886,NaN,"POLYGON Z ((-99.27973 19.44320 0.00000, -99.27..."
4,15,098,0000,0981001819000000,192648262609917139131,NaN,"POLYGON Z ((-99.28698 19.44671 0.00000, -99.28..."
...,...,...,...,...,...,...,...
125145,15,098,0000,0980919611000000,NaN,14 _ Ver curtnotas,"POLYGON Z ((-99.25794 19.44273 0.00000, -99.25..."
125146,15,098,0000,0980308811000000,192700419009914229387,NaN,"POLYGON Z ((-99.23960 19.45020 0.00000, -99.23..."
125147,15,098,0000,0980832747000000,193013257609914046937,NaN,"POLYGON Z ((-99.23457 19.50359 0.00000, -99.23..."
125148,15,098,0000,0980377409000000,192519218009913521489,NaN,"POLYGON Z ((-99.23101 19.42200 0.00000, -99.23..."


In [5]:
casas_filtro=gpd.read_file(r"H:\.shortcut-targets-by-id\19j98C-PJkrk3gkgU7V03dLiJ4Gn-hnNe\geoshaps\naucalpan (1)\nueva_rotacion\Naucalpan_casas_nueva_rotacion.shp")

In [ ]:
casas_filtro["ID_casas"]=casas_filtro.index
casas_filtro

,status,geometry,ID_casas
0,correcto,"POLYGON ((-11066297.517 2206681.680, -11066296...",0
1,correcto,"POLYGON ((-11065706.721 2207306.024, -11065697...",1
2,correcto,"POLYGON ((-11065643.087 2207240.371, -11065641...",2
3,correcto,"POLYGON ((-11065605.148 2207275.495, -11065599...",3
4,correcto,"POLYGON ((-11065491.476 2206352.778, -11065474...",4
...,...,...,...
254528,correcto,"POLYGON ((-11043758.600 2210340.079, -11043757...",254528
254529,correcto,"POLYGON ((-11043763.319 2210358.494, -11043761...",254529
254530,correcto,"POLYGON ((-11043791.376 2210346.050, -11043789...",254530
254531,correcto,"POLYGON ((-11043762.495 2210339.416, -11043760...",254531


In [5]:
# CURT.geometry.value_counts()
casas_filtro.geometry.value_counts()

POLYGON ((-11105767.238 2135577.743, -11105746...    1
POLYGON ((-11105762.935 2135576.184, -11105753...    1
POLYGON ((-11105754.277 2135576.711, -11105741...    1
POLYGON ((-11105763.596 2135577.521, -11105747...    1
POLYGON ((-11105751.911 2135425.039, -11105739...    1
                                                    ..
POLYGON ((-11088640.599 2136511.008, -11088633...    1
POLYGON ((-11088590.924 2136573.546, -11088580...    1
POLYGON ((-11088763.470 2136449.699, -11088744...    1
POLYGON ((-11088905.958 2134917.172, -11088896...    1
POLYGON ((-11088580.218 2136547.235, -11088561...    1
Name: geometry, Length: 29208, dtype: int64

In [8]:
#Se revisa el CRS de cada base para poder hacer el cruce entre ellas
print("crs CURT: ",CURT.crs)
print("crs casas: ",casas_filtro.crs)

crs CURT:  EPSG:3857
crs casas:  EPSG:3857


In [7]:
CURT=CURT.set_crs(6364,allow_override=True)
# CURT.geometry.head()
CURT=CURT.to_crs(3857)
CURT.geometry

0       POLYGON ((-11095423.459 2142567.445, -11095438...
1       POLYGON ((-11097546.795 2136392.532, -11097560...
2       POLYGON ((-11091457.811 2134957.209, -11091505...
3       POLYGON ((-11094920.538 2133724.816, -11094920...
4       POLYGON ((-11094687.882 2133853.801, -11094729...
                              ...                        
7197    POLYGON ((-11095751.202 2136305.140, -11095758...
7198    POLYGON ((-11096065.668 2136478.225, -11096066...
7199    POLYGON ((-11095342.589 2136502.004, -11095341...
7200    POLYGON ((-11095842.684 2136009.880, -11095841...
7201    POLYGON ((-11096034.591 2134820.915, -11096034...
Name: geometry, Length: 7202, dtype: geometry

In [124]:
casas_filtro=casas_filtro.set_crs(3857,allow_override=True)
# casas_filtro.crs
# # CURT.geometry.head()
# casas_filtro.to_crs(3857)
casas_filtro.geometry

0        POLYGON ((-11105767.238 2135577.743, -11105746...
1        POLYGON ((-11105772.754 2135599.757, -11105751...
2        POLYGON ((-11105762.935 2135576.184, -11105753...
3        POLYGON ((-11105760.686 2135571.164, -11105749...
4        POLYGON ((-11105754.277 2135576.711, -11105741...
                               ...                        
29203    POLYGON ((-11088683.142 2135199.640, -11088676...
29204    POLYGON ((-11088600.094 2136546.559, -11088581...
29205    POLYGON ((-11088640.599 2136511.008, -11088633...
29206    POLYGON ((-11088590.924 2136573.546, -11088580...
29207    POLYGON ((-11088580.218 2136547.235, -11088561...
Name: geometry, Length: 29208, dtype: geometry

In [9]:
casas_filtro=obtener_curt(casas_filtro,'geometry')
casas_filtro

C:\Users\mfpen\Documents\Repositorios\geoloc2\Detecciondeterrenos\codigos.py:347: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  data['centroid'] = data[geom_col].to_crs(4326).centroid


,clase_dete,conf,area,status,geometry,ID_casas,CURT_f
0,establecimiento,0.48738,208.166051,correcto,"POLYGON ((-11105767.238 2135577.743, -11105746...",0,18500780699945528681
1,casas,0.63394,422.365900,correcto,"POLYGON ((-11105772.754 2135599.757, -11105751...",1,18500820519945529679
2,establecimiento,0.27702,56.315453,correcto,"POLYGON ((-11105762.935 2135576.184, -11105753...",2,18500763379945529792
3,establecimiento,0.32621,160.901295,correcto,"POLYGON ((-11105760.686 2135571.164, -11105749...",3,18500751819945528880
4,establecimiento,0.47255,217.216749,correcto,"POLYGON ((-11105754.277 2135576.711, -11105741...",4,18500756269945525823
...,...,...,...,...,...,...,...
29203,casas,0.74758,120.675972,correcto,"POLYGON ((-11088683.142 2135199.640, -11088676...",29203,18495609089936406801
29204,establecimiento,0.27273,290.627965,correcto,"POLYGON ((-11088600.094 2136546.559, -11088581...",29204,18503728779936376677
29205,establecimiento,0.55595,231.295612,correcto,"POLYGON ((-11088640.599 2136511.008, -11088633...",29205,18503596579936391706
29206,establecimiento,0.28596,464.602490,correcto,"POLYGON ((-11088590.924 2136573.546, -11088580...",29206,18503777899936373374


In [10]:
CURT.shape, casas_filtro.shape

((7202, 8), (29208, 7))

In [11]:
cruce_curt=gpd.sjoin(CURT,casas_filtro)
cruce_curt

,cve_ent,cve_mun,cve_loc,id_cat,proceso,curt,notas,geometry,index_right,clase_dete,conf,area,status,ID_casas,CURT_f
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",21804,casas,0.71761,343.926519,correcto,21804,18534524149940231040
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22066,casas,0.23445,48.270389,correcto,22066,18534528279940228467
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22610,establecimiento,0.65228,144.811166,correcto,22610,18534469199940170361
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22331,establecimiento,0.32949,42.236590,correcto,22331,18534625739940206472
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22874,establecimiento,0.72347,117.659072,correcto,22874,18534339529940157520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7169,15,060,0000,0600603528000000,G,185034904609942005998,NaN,"POLYGON ((-11098581.906 2136457.527, -11098572...",11021,establecimiento,0.71012,121.681605,correcto,11021,18503508239942007285
7170,15,060,0000,0600603523000000,G,185035144409942010934,NaN,"POLYGON ((-11098578.841 2136475.333, -11098588...",11021,establecimiento,0.71012,121.681605,correcto,11021,18503508239942007285
7171,15,060,0000,0600603511000000,G,185034984509942008777,NaN,"POLYGON ((-11098572.171 2136470.109, -11098581...",11021,establecimiento,0.71012,121.681605,correcto,11021,18503508239942007285
7172,15,060,0000,0600603533000000,G,185035449609942006708,NaN,"POLYGON ((-11098562.436 2136482.692, -11098572...",11021,establecimiento,0.71012,121.681605,correcto,11021,18503508239942007285


In [12]:
cruce_curt['CURT_si']=1
cruce_curt

,cve_ent,cve_mun,cve_loc,id_cat,proceso,curt,notas,geometry,index_right,clase_dete,conf,area,status,ID_casas,CURT_f,CURT_si
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",21804,casas,0.71761,343.926519,correcto,21804,18534524149940231040,1
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22066,casas,0.23445,48.270389,correcto,22066,18534528279940228467,1
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22610,establecimiento,0.65228,144.811166,correcto,22610,18534469199940170361,1
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22331,establecimiento,0.32949,42.236590,correcto,22331,18534625739940206472,1
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22874,establecimiento,0.72347,117.659072,correcto,22874,18534339529940157520,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7169,15,060,0000,0600603528000000,G,185034904609942005998,NaN,"POLYGON ((-11098581.906 2136457.527, -11098572...",11021,establecimiento,0.71012,121.681605,correcto,11021,18503508239942007285,1
7170,15,060,0000,0600603523000000,G,185035144409942010934,NaN,"POLYGON ((-11098578.841 2136475.333, -11098588...",11021,establecimiento,0.71012,121.681605,correcto,11021,18503508239942007285,1
7171,15,060,0000,0600603511000000,G,185034984509942008777,NaN,"POLYGON ((-11098572.171 2136470.109, -11098581...",11021,establecimiento,0.71012,121.681605,correcto,11021,18503508239942007285,1
7172,15,060,0000,0600603533000000,G,185035449609942006708,NaN,"POLYGON ((-11098562.436 2136482.692, -11098572...",11021,establecimiento,0.71012,121.681605,correcto,11021,18503508239942007285,1


In [13]:
cruce_curt.geometry.unique()

<GeometryArray>
[<POLYGON ((-11095423.459 2142567.445, -11095438.596 2142555.357, -11095564.4...>,
 <POLYGON ((-11095261.341 2142655.014, -11095282.184 2142639.297, -11095287.8...>,
 <POLYGON ((-11091457.811 2134957.209, -11091505.231 2134938.926, -11091597.5...>,
 <POLYGON ((-11094920.538 2133724.816, -11094920.508 2133724.814, -11094899.5...>,
 <POLYGON ((-11094997.06 2133681.208, -11094987.016 2133680.354, -11094992.86...>,
 <POLYGON ((-11094995.79 2133556.844, -11094995.657 2133558.372, -11094992.86...>,
 <POLYGON ((-11095043.995 2133735.321, -11095012.389 2133732.819, -11094992.4...>,
 <POLYGON ((-11094923.45 2133747.084, -11094923.437 2133747.083, -11094920.53...>,
 <POLYGON ((-11094687.882 2133853.801, -11094729.881 2133850.857, -11094731.3...>,
 <POLYGON ((-11094685.829 2133822.302, -11094760.462 2133825.815, -11094761.8...>,
 ...
 <POLYGON ((-11095283.446 2137067.108, -11095290.231 2137059.039, -11095313.9...>,
 <POLYGON ((-11091321.592 2138809.539, -11091369.645 2138787.436, 

In [14]:
no_cruzanxcurt=CURT[~(CURT.geometry.isin(cruce_curt['geometry']))]
no_cruzanxcurt

,cve_ent,cve_mun,cve_loc,id_cat,proceso,curt,notas,geometry
1,15,060,0000,0600602005000000,G,185034596809941282529,NaN,"POLYGON ((-11097546.795 2136392.532, -11097560..."
5,15,060,0000,0600100624000000,G,185040396409940285814,NaN,"POLYGON ((-11095724.396 2136641.874, -11095728..."
14,15,060,0000,0600602405000000,G,185022164109941313416,NaN,"POLYGON ((-11097599.727 2136084.388, -11097562..."
15,15,060,0000,0600302019000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095061.502 2142416.953, -11095031..."
16,15,060,0000,0600302062000000,G,185338606509940069324,NaN,"POLYGON ((-11095061.502 2142416.953, -11095101..."
...,...,...,...,...,...,...,...,...
7177,15,060,0000,0600121710000000,G,184939135409940476787,NaN,"POLYGON ((-11096306.672 2134633.621, -11096306..."
7178,15,060,0000,0600121711000000,G,184939622909940476569,NaN,"POLYGON ((-11096306.003 2134649.558, -11096327..."
7193,15,060,0000,0600105681000000,G,185031747409940385550,NaN,"POLYGON ((-11096027.630 2136355.964, -11096029..."
7199,15,060,0000,0600104871000000,G,185035785209940161798,NaN,"POLYGON ((-11095342.589 2136502.004, -11095341..."


In [15]:
no_cruzanxcurt['CURT_si']=0
no_cruzanxcurt

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,cve_ent,cve_mun,cve_loc,id_cat,proceso,curt,notas,geometry,CURT_si
1,15,060,0000,0600602005000000,G,185034596809941282529,NaN,"POLYGON ((-11097546.795 2136392.532, -11097560...",0
5,15,060,0000,0600100624000000,G,185040396409940285814,NaN,"POLYGON ((-11095724.396 2136641.874, -11095728...",0
14,15,060,0000,0600602405000000,G,185022164109941313416,NaN,"POLYGON ((-11097599.727 2136084.388, -11097562...",0
15,15,060,0000,0600302019000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095061.502 2142416.953, -11095031...",0
16,15,060,0000,0600302062000000,G,185338606509940069324,NaN,"POLYGON ((-11095061.502 2142416.953, -11095101...",0
...,...,...,...,...,...,...,...,...,...
7177,15,060,0000,0600121710000000,G,184939135409940476787,NaN,"POLYGON ((-11096306.672 2134633.621, -11096306...",0
7178,15,060,0000,0600121711000000,G,184939622909940476569,NaN,"POLYGON ((-11096306.003 2134649.558, -11096327...",0
7193,15,060,0000,0600105681000000,G,185031747409940385550,NaN,"POLYGON ((-11096027.630 2136355.964, -11096029...",0
7199,15,060,0000,0600104871000000,G,185035785209940161798,NaN,"POLYGON ((-11095342.589 2136502.004, -11095341...",0


In [16]:
cruce_curt.shape, no_cruzanxcurt.shape

((23163, 16), (1550, 9))

In [17]:
cruce_final_curt=pd.concat([cruce_curt,no_cruzanxcurt])
cruce_final_curt

,cve_ent,cve_mun,cve_loc,id_cat,proceso,curt,notas,geometry,index_right,clase_dete,conf,area,status,ID_casas,CURT_f,CURT_si
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",21804.0,casas,0.71761,343.926519,correcto,21804.0,18534524149940231040,1
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22066.0,casas,0.23445,48.270389,correcto,22066.0,18534528279940228467,1
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22610.0,establecimiento,0.65228,144.811166,correcto,22610.0,18534469199940170361,1
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22331.0,establecimiento,0.32949,42.236590,correcto,22331.0,18534625739940206472,1
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22874.0,establecimiento,0.72347,117.659072,correcto,22874.0,18534339529940157520,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7177,15,060,0000,0600121710000000,G,184939135409940476787,NaN,"POLYGON ((-11096306.672 2134633.621, -11096306...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7178,15,060,0000,0600121711000000,G,184939622909940476569,NaN,"POLYGON ((-11096306.003 2134649.558, -11096327...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7193,15,060,0000,0600105681000000,G,185031747409940385550,NaN,"POLYGON ((-11096027.630 2136355.964, -11096029...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7199,15,060,0000,0600104871000000,G,185035785209940161798,NaN,"POLYGON ((-11095342.589 2136502.004, -11095341...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [18]:
#Para detectar duplicados x geometría
geo_dupli=cruce_final_curt.geometry.value_counts()[cruce_final_curt.geometry.value_counts()>1].index
cruce_final_curt.loc[cruce_final_curt.geometry.isin(geo_dupli),"dupGEO"]=1
cruce_final_curt.loc[cruce_final_curt.dupGEO != 1, "dupGEO"]=0
cruce_final_curt


,cve_ent,cve_mun,cve_loc,id_cat,proceso,curt,notas,geometry,index_right,clase_dete,conf,area,status,ID_casas,CURT_f,CURT_si,dupGEO
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",21804.0,casas,0.71761,343.926519,correcto,21804.0,18534524149940231040,1,1.0
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22066.0,casas,0.23445,48.270389,correcto,22066.0,18534528279940228467,1,1.0
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22610.0,establecimiento,0.65228,144.811166,correcto,22610.0,18534469199940170361,1,1.0
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22331.0,establecimiento,0.32949,42.236590,correcto,22331.0,18534625739940206472,1,1.0
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22874.0,establecimiento,0.72347,117.659072,correcto,22874.0,18534339529940157520,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7177,15,060,0000,0600121710000000,G,184939135409940476787,NaN,"POLYGON ((-11096306.672 2134633.621, -11096306...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
7178,15,060,0000,0600121711000000,G,184939622909940476569,NaN,"POLYGON ((-11096306.003 2134649.558, -11096327...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
7193,15,060,0000,0600105681000000,G,185031747409940385550,NaN,"POLYGON ((-11096027.630 2136355.964, -11096029...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0
7199,15,060,0000,0600104871000000,G,185035785209940161798,NaN,"POLYGON ((-11095342.589 2136502.004, -11095341...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0


In [19]:
#Para detectar duplicados x curt
curt_dupli=cruce_final_curt.curt.value_counts()[cruce_final_curt.curt.value_counts()>1].index
cruce_final_curt.loc[cruce_final_curt.curt.isin(curt_dupli),"dupCURT"]=1
cruce_final_curt.loc[cruce_final_curt.dupCURT != 1, "dupCURT"]=0
cruce_final_curt

,cve_ent,cve_mun,cve_loc,id_cat,proceso,curt,notas,geometry,index_right,clase_dete,conf,area,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",21804.0,casas,0.71761,343.926519,correcto,21804.0,18534524149940231040,1,1.0,0.0
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22066.0,casas,0.23445,48.270389,correcto,22066.0,18534528279940228467,1,1.0,0.0
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22610.0,establecimiento,0.65228,144.811166,correcto,22610.0,18534469199940170361,1,1.0,0.0
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22331.0,establecimiento,0.32949,42.236590,correcto,22331.0,18534625739940206472,1,1.0,0.0
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22874.0,establecimiento,0.72347,117.659072,correcto,22874.0,18534339529940157520,1,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7177,15,060,0000,0600121710000000,G,184939135409940476787,NaN,"POLYGON ((-11096306.672 2134633.621, -11096306...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0
7178,15,060,0000,0600121711000000,G,184939622909940476569,NaN,"POLYGON ((-11096306.003 2134649.558, -11096327...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0
7193,15,060,0000,0600105681000000,G,185031747409940385550,NaN,"POLYGON ((-11096027.630 2136355.964, -11096029...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0
7199,15,060,0000,0600104871000000,G,185035785209940161798,NaN,"POLYGON ((-11095342.589 2136502.004, -11095341...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0


In [ ]:
print('Columas de casas: ', cruce_final_curt.columns)
if input('Quiere crear la columna ID_casas') in ['1','si']:
    cruce_final_curt.reset_index(drop=False, inplace=True)
    cruce_final_curt.rename(columns={'index':'ID_casas'},inplace=True)

In [21]:
casas_nocruzan = casas_filtro[~casas_filtro.ID_casas.isin(cruce_curt.ID_casas.unique())]
casas_nocruzan['YOLO_si'] = 0
casas_nocruzan.tail(2)

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,clase_dete,conf,area,status,geometry,ID_casas,CURT_f,YOLO_si
29206,establecimiento,0.28596,464.602490,correcto,"POLYGON ((-11088590.924 2136573.546, -11088580...",29206,18503777899936373374,0
29207,casas,0.55884,162.912562,correcto,"POLYGON ((-11088580.218 2136547.235, -11088561...",29207,18503736079936371225,0


In [22]:
cruce_final_curt.shape, casas_nocruzan.shape

((24713, 18), (18636, 8))

In [23]:
cruce_final =pd.concat([cruce_final_curt,casas_nocruzan])
cruce_final

,cve_ent,cve_mun,cve_loc,id_cat,proceso,curt,notas,geometry,index_right,clase_dete,conf,area,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT,YOLO_si
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",21804.0,casas,0.71761,343.926519,correcto,21804.0,18534524149940231040,1.0,1.0,0.0,NaN
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22066.0,casas,0.23445,48.270389,correcto,22066.0,18534528279940228467,1.0,1.0,0.0,NaN
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22610.0,establecimiento,0.65228,144.811166,correcto,22610.0,18534469199940170361,1.0,1.0,0.0,NaN
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22331.0,establecimiento,0.32949,42.236590,correcto,22331.0,18534625739940206472,1.0,1.0,0.0,NaN
0,15,060,0000,0600302005000000,G,NaN,14 _ Ver curtnotas,"POLYGON ((-11095423.459 2142567.445, -11095438...",22874.0,establecimiento,0.72347,117.659072,correcto,22874.0,18534339529940157520,1.0,1.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11088683.142 2135199.640, -11088676...",NaN,casas,0.74758,120.675972,correcto,29203.0,18495609089936406801,NaN,NaN,NaN,0.0
29204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11088600.094 2136546.559, -11088581...",NaN,establecimiento,0.27273,290.627965,correcto,29204.0,18503728779936376677,NaN,NaN,NaN,0.0
29205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11088640.599 2136511.008, -11088633...",NaN,establecimiento,0.55595,231.295612,correcto,29205.0,18503596579936391706,NaN,NaN,NaN,0.0
29206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11088590.924 2136573.546, -11088580...",NaN,establecimiento,0.28596,464.602490,correcto,29206.0,18503777899936373374,NaN,NaN,NaN,0.0


In [24]:
cruce_final.loc[cruce_final.YOLO_si.isna(),"YOLO_si"]=1

In [25]:
cruce_final.rename(columns={'index_right':'ID_curt'},inplace=True)

Cruce casas x industrias

In [26]:
path_industrial = r"H:\.shortcut-targets-by-id\19j98C-PJkrk3gkgU7V03dLiJ4Gn-hnNe\geoshaps\Ixtapan_Sal\industria\Ixtapan_industrias_filtro.shp"
industrias = gpd.read_file(path_industrial)

In [27]:
cruce_final.shape, industrias.shape

((43349, 19), (666, 11))

In [28]:
cruce_casasxind = gpd.sjoin(industrias,cruce_final)
cruce_casasxind

,puntos,zona,area_left,rectangulo,Medida_lx,Medida_ly,Max_xy,Min_xy,f_lado,f_area,...,clase_dete,conf,area_right,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT,YOLO_si
0,149,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,establecimiento,0.81010,222.244914,correcto,8620.0,18513662109942335296,NaN,NaN,NaN,0.0
0,149,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,establecimiento,0.58994,502.816548,correcto,8614.0,18513711809942336682,NaN,NaN,NaN,0.0
0,149,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,establecimiento,0.20555,524.940476,correcto,8613.0,18513753159942340226,NaN,NaN,NaN,0.0
1,89,02,1080.604247,1137.882352,44.4497,25.7522,44.4497,25.7522,1.726054,0.949663,...,establecimiento,0.70573,896.019089,correcto,4469.0,18515161659943278317,NaN,NaN,NaN,0.0
1,89,02,1080.604247,1137.882352,44.4497,25.7522,44.4497,25.7522,1.726054,0.949663,...,establecimiento,0.65959,154.867497,correcto,4596.0,18515198549943272513,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,338,09,463.425800,626.140814,14.3218,43.2013,43.2013,14.3218,3.016471,0.740130,...,establecimiento,0.71699,422.365900,correcto,1062.0,18502579659944516443,1.0,1.0,1.0,1.0
665,264,09,480.625273,721.417894,17.8614,39.7185,39.7185,17.8614,2.223706,0.666223,...,establecimiento,0.67255,99.557677,correcto,709.0,18503076299945091265,NaN,NaN,NaN,0.0
665,264,09,480.625273,721.417894,17.8614,39.7185,39.7185,17.8614,2.223706,0.666223,...,establecimiento,0.82027,130.732302,correcto,710.0,18503109779945092268,NaN,NaN,NaN,0.0
665,264,09,480.625273,721.417894,17.8614,39.7185,39.7185,17.8614,2.223706,0.666223,...,establecimiento,0.45620,844.731801,correcto,656.0,18503127659945095547,NaN,NaN,NaN,0.0


In [29]:
cruce_casasxind['Ind_si']=1
cruce_casasxind

,puntos,zona,area_left,rectangulo,Medida_lx,Medida_ly,Max_xy,Min_xy,f_lado,f_area,...,conf,area_right,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT,YOLO_si,Ind_si
0,149,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,0.81010,222.244914,correcto,8620.0,18513662109942335296,NaN,NaN,NaN,0.0,1
0,149,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,0.58994,502.816548,correcto,8614.0,18513711809942336682,NaN,NaN,NaN,0.0,1
0,149,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,0.20555,524.940476,correcto,8613.0,18513753159942340226,NaN,NaN,NaN,0.0,1
1,89,02,1080.604247,1137.882352,44.4497,25.7522,44.4497,25.7522,1.726054,0.949663,...,0.70573,896.019089,correcto,4469.0,18515161659943278317,NaN,NaN,NaN,0.0,1
1,89,02,1080.604247,1137.882352,44.4497,25.7522,44.4497,25.7522,1.726054,0.949663,...,0.65959,154.867497,correcto,4596.0,18515198549943272513,NaN,NaN,NaN,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664,338,09,463.425800,626.140814,14.3218,43.2013,43.2013,14.3218,3.016471,0.740130,...,0.71699,422.365900,correcto,1062.0,18502579659944516443,1.0,1.0,1.0,1.0,1
665,264,09,480.625273,721.417894,17.8614,39.7185,39.7185,17.8614,2.223706,0.666223,...,0.67255,99.557677,correcto,709.0,18503076299945091265,NaN,NaN,NaN,0.0,1
665,264,09,480.625273,721.417894,17.8614,39.7185,39.7185,17.8614,2.223706,0.666223,...,0.82027,130.732302,correcto,710.0,18503109779945092268,NaN,NaN,NaN,0.0,1
665,264,09,480.625273,721.417894,17.8614,39.7185,39.7185,17.8614,2.223706,0.666223,...,0.45620,844.731801,correcto,656.0,18503127659945095547,NaN,NaN,NaN,0.0,1


In [30]:
no_cruzanxInd=industrias[~(industrias.geometry.isin(cruce_casasxind['geometry']))]
no_cruzanxInd.tail(2)

,puntos,zona,area,rectangulo,Medida_lx,Medida_ly,Max_xy,Min_xy,f_lado,f_area,geometry
616,788,09,1036.208248,2095.615786,56.6767,36.9749,56.6767,36.9749,1.532843,0.494465,"POLYGON ((-11098462.080 2128663.912, -11098456..."
617,1059,09,1082.846819,2641.839049,97.1913,27.1819,97.1913,27.1819,3.575589,0.409884,"POLYGON ((-11098510.879 2128684.712, -11098458..."


In [31]:
no_cruzanxInd['Ind_si']=0
no_cruzanxInd.tail(2)

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,puntos,zona,area,rectangulo,Medida_lx,Medida_ly,Max_xy,Min_xy,f_lado,f_area,geometry,Ind_si
616,788,09,1036.208248,2095.615786,56.6767,36.9749,56.6767,36.9749,1.532843,0.494465,"POLYGON ((-11098462.080 2128663.912, -11098456...",0
617,1059,09,1082.846819,2641.839049,97.1913,27.1819,97.1913,27.1819,3.575589,0.409884,"POLYGON ((-11098510.879 2128684.712, -11098458...",0


In [32]:
cruce_casasxind.shape, no_cruzanxInd.shape

((3858, 31), (40, 12))

In [33]:
crucexInd=pd.concat([cruce_casasxind,no_cruzanxInd])
crucexInd

,puntos,zona,area_left,rectangulo,Medida_lx,Medida_ly,Max_xy,Min_xy,f_lado,f_area,...,area_right,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT,YOLO_si,Ind_si,area
0,149,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,222.244914,correcto,8620.0,18513662109942335296,NaN,NaN,NaN,0.0,1,NaN
0,149,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,502.816548,correcto,8614.0,18513711809942336682,NaN,NaN,NaN,0.0,1,NaN
0,149,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,524.940476,correcto,8613.0,18513753159942340226,NaN,NaN,NaN,0.0,1,NaN
1,89,02,1080.604247,1137.882352,44.4497,25.7522,44.4497,25.7522,1.726054,0.949663,...,896.019089,correcto,4469.0,18515161659943278317,NaN,NaN,NaN,0.0,1,NaN
1,89,02,1080.604247,1137.882352,44.4497,25.7522,44.4497,25.7522,1.726054,0.949663,...,154.867497,correcto,4596.0,18515198549943272513,NaN,NaN,NaN,0.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
582,2037,01,NaN,13932.717368,121.3612,114.5437,121.3612,114.5437,1.059519,0.538523,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,7503.087828
583,359,01,NaN,1465.064960,46.2086,31.6484,46.2086,31.6484,1.460061,0.517228,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,757.772910
615,623,09,NaN,1639.389766,27.7846,58.9836,58.9836,27.7846,2.122888,0.427024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,700.058549
616,788,09,NaN,2095.615786,56.6767,36.9749,56.6767,36.9749,1.532843,0.494465,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1036.208248


In [34]:
nocruzan_casas =cruce_final[~cruce_final.ID_casas.isin(cruce_casasxind.ID_casas.unique())]
nocruzan_casas['Induxcasa'] = 0
nocruzan_casas.tail(2)

c:\Users\mfpen\anaconda3\envs\YOLO\lib\site-packages\geopandas\geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,cve_ent,cve_mun,cve_loc,id_cat,proceso,curt,notas,geometry,ID_curt,clase_dete,conf,area,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT,YOLO_si,Induxcasa
29206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11088590.924 2136573.546, -11088580...",NaN,establecimiento,0.28596,464.602490,correcto,29206.0,18503777899936373374,NaN,NaN,NaN,0.0,0
29207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-11088580.218 2136547.235, -11088561...",NaN,casas,0.55884,162.912562,correcto,29207.0,18503736079936371225,NaN,NaN,NaN,0.0,0


In [35]:
cruce_final_ind =pd.concat([crucexInd,nocruzan_casas])
cruce_final_ind

,puntos,zona,area_left,rectangulo,Medida_lx,Medida_ly,Max_xy,Min_xy,f_lado,f_area,...,status,ID_casas,CURT_f,CURT_si,dupGEO,dupCURT,YOLO_si,Ind_si,area,Induxcasa
0,149.0,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,correcto,8620.0,18513662109942335296,NaN,NaN,NaN,0.0,1.0,NaN,NaN
0,149.0,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,correcto,8614.0,18513711809942336682,NaN,NaN,NaN,0.0,1.0,NaN,NaN
0,149.0,02,362.867994,504.943293,27.6806,18.2425,27.6806,18.2425,1.517369,0.718631,...,correcto,8613.0,18513753159942340226,NaN,NaN,NaN,0.0,1.0,NaN,NaN
1,89.0,02,1080.604247,1137.882352,44.4497,25.7522,44.4497,25.7522,1.726054,0.949663,...,correcto,4469.0,18515161659943278317,NaN,NaN,NaN,0.0,1.0,NaN,NaN
1,89.0,02,1080.604247,1137.882352,44.4497,25.7522,44.4497,25.7522,1.726054,0.949663,...,correcto,4596.0,18515198549943272513,NaN,NaN,NaN,0.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,correcto,29203.0,18495609089936406801,NaN,NaN,NaN,0.0,NaN,120.675972,0.0
29204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,correcto,29204.0,18503728779936376677,NaN,NaN,NaN,0.0,NaN,290.627965,0.0
29205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,correcto,29205.0,18503596579936391706,NaN,NaN,NaN,0.0,NaN,231.295612,0.0
29206,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,correcto,29206.0,18503777899936373374,NaN,NaN,NaN,0.0,NaN,464.602490,0.0


In [36]:
cruce_final_ind.rename(columns={"index_right":"ID_ind"},inplace=True)
cruce_final_ind.columns

Index(['puntos', 'zona', 'area_left', 'rectangulo', 'Medida_lx', 'Medida_ly',
       'Max_xy', 'Min_xy', 'f_lado', 'f_area', 'geometry', 'ID_ind', 'cve_ent',
       'cve_mun', 'cve_loc', 'id_cat', 'proceso', 'curt', 'notas', 'ID_curt',
       'clase_dete', 'conf', 'area_right', 'status', 'ID_casas', 'CURT_f',
       'CURT_si', 'dupGEO', 'dupCURT', 'YOLO_si', 'Ind_si', 'area',
       'Induxcasa'],
      dtype='object')

In [37]:
cruce_final_ind.to_file(r"D:\Secretaria\cruces_bases\limpiezas_shp_base\Ixtapan\ultimas_versiones/Ixtapan_casas_Indxcurt.shp")